In [ ]:
is_simulation = True

In [ ]:
import numpy as np
import sys
sys.path.insert(0, '..')
from bundle import FPGA_config, evaluate

if is_simulation:
    import matplotlib.pyplot as plt
    %matplotlib inline
    from bundle.simu import Simu
    from pyclk import Trace
else:
    from bundle.pynq import PYNQ
    from pynq import Xlnk, Overlay
    xlnk = Xlnk()
    overlay = Overlay('/home/xilinx/overlays/bundle/design_1.bit')

In [ ]:
iter_nb = 1
mem_nb = iter_nb * 3
ddr2fpga_nb = 1
fpga2ddr_nb = 1
mem_depth = 1024
add_nb = iter_nb
mul_nb = iter_nb

fpga_config = FPGA_config(ddr2fpga_nb, fpga2ddr_nb, iter_nb, mem_nb, mem_depth, add_nb, mul_nb)

if is_simulation:
    fpga = Simu(fpga_config)
else:
    fpga = PYNQ(fpga_config, overlay)

In [ ]:
if is_simulation:
    #fpga.set_cycle_nb(1000)
    trace = Trace()
    for i in range(fpga_config.config['ddr2fpga_nb']):
        #trace.add(fpga.u_ddr2fpga[i].r_state)
        #trace.add(fpga.u_ddr2fpga[i].i_data_nb)
        #trace.add(fpga.u_ddr2fpga[i].r_wena)
        #trace.add(fpga.u_ddr2fpga[i].r_ptr_i)
        trace.add(fpga.u_ddr2fpga[i].o_mem_din)
        trace.add(fpga.s_ddr2fpga_mem_i[i])
    for i in range(fpga_config.config['mem_nb']):
        #trace.add(fpga.u_mem[i].i_addr)
        trace.add(fpga.u_mem[i].o_dout)
        trace.add(fpga.u_mem[i].i_din)
        trace.add(fpga.s_mem_dout[i])
        #trace.add(fpga.u_mem[i].i_wena)
        #trace.add(fpga.s_mem_din[i])
    for i in range(fpga_config.config['iter_nb']):
        trace.add(fpga.s_iter_rmem0_i[i])
        trace.add(fpga.s_iter_rmem1_i[i])
        #trace.add(fpga.u_iter[i].o_done)
        #trace.add(fpga.u_iter[i].i_ack)
        #trace.add(fpga.u_iter[i].i_data_nb)
    for i in range(fpga_config.config['func_nb']):
        trace.add(fpga.u_func[i].i_arg0)
        trace.add(fpga.u_func[i].i_arg1)
        trace.add(fpga.u_func[i].i_arg_valid)
        trace.add(fpga.u_func[i].o_res)
        trace.add(fpga.u_func[i].o_res_valid)
    for i in range(fpga_config.config['fpga2ddr_nb']):
        trace.add(fpga.u_fpga2ddr[i].i_data_nb)
        trace.add(fpga.u_fpga2ddr[i].o_done)
    fpga.set_trace(trace)

In [ ]:
size = 10_000
a0 = np.arange(size, dtype=np.uint64)
a1 = np.ones_like(a0)
a2 = np.zeros_like(a0)

#expr = '(a0 + a1 * (a0 + a2)) * a1 + a1'
expr = 'a0 + a1'

In [ ]:
res = evaluate(expr, fpga, debug=True)

In [ ]:
res

In [ ]:
# FPGA runs at 100 MHz
print('In FPGA:')
print(f'{fpga.time * 1e6 / 1e8} us')

In [ ]:
cexpr = compile(expr, 'fakemodule', 'exec')
print('In CPU:')
%timeit exec(cexpr)

In [ ]:
#res_true = bd.evaluate(expr, toFpga=False)
res_true = eval(expr)
res_true

In [ ]:
if np.array_equal(res, res_true):
    print('Success!')
else:
    print('Failed!')

In [ ]:
res = np.array(res)
diff_idx = np.where(res != res_true)
plt.figure(figsize=(15, 5))
plt.scatter(np.arange(res_true.size), res_true, c='b')
plt.scatter(diff_idx, res[diff_idx], c='r', label='FPGA')
plt.scatter(diff_idx, res_true[diff_idx], c='g', label='NumPy')
#plt.scatter(diff_idx, z_true[diff_idx]-z[diff_idx], c='r')
plt.xlim(0, size)
plt.ylim(1, size+1)
plt.legend(loc='upper left')
plt.title('Differences')
plt.show()

In [ ]:
res[diff_idx]

In [ ]:
res_true[diff_idx]

In [ ]:
trace.plot()